In [2]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu
import pandas as pd
import numpy as np
from time import time
import keras
from keras.utils import np_utils
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
Orginal_Signatures = "F:\\Aac dataset\\Dataset\\real/"
Fake_Signatures = "F:\\Aac dataset\\Dataset\\forged/"

In [4]:
def ColorToGray(img):
    Grayimage = np.zeros((img.shape[0], img.shape[1]))
    for i in range(len(img)):
        for j in range(len(img[i])):
            Grayimage[i][j] = np.average(img[i][j])
    return Grayimage
def GrayToBinary(img):
    blur_radius = 0.8
    img = ndimage.gaussian_filter(img, blur_radius)
    ThresholdFreq = threshold_otsu(img)
    BinaryImg = img > ThresholdFreq
    BinaryImg = np.logical_not(BinaryImg)
    return BinaryImg
def PreProccesing(path, img=None, display=True):
    if img is None:
        img = mpimg.imread(path)
    if display:
        plt.imshow(img)
        plt.show()
    Grey = ColorToGray(img)
    if display:
        plt.imshow(Grey, cmap = matplotlib.cm.Greys_r)
        plt.show()
    BinaryImg = GrayToBinary(Grey)
    if display:
        plt.imshow(BinaryImg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    r, c = np.where(BinaryImg==1)
    SignatureImage = BinaryImg[r.min(): r.max(), c.min(): c.max()]
    if display:
        plt.imshow(SignatureImage, cmap = matplotlib.cm.Greys_r)
        plt.show()
    return SignatureImage

# Feature Extraction:

In [5]:
def Calculating_Ratio(img):
    pixels = 0
    for i in range(len(img)):
        for j in range(len(img[0])):
            if img[i][j]==True:
                pixels += 1
    total_pixels = img.shape[0] * img.shape[1]
    return pixels/total_pixels
def Calculating_EccentricityAndSolidity(img):
    region = regionprops(img.astype("int8"))
    return region[0].eccentricity, region[0].solidity
def Calculating_SkewKurtosis(img):
    h,w = img.shape
    x = range(w)
    y = range(h)
    xp = np.sum(img,axis=0)
    yp = np.sum(img,axis=1)
    cx = np.sum(x*xp)/np.sum(xp)
    cy = np.sum(y*yp)/np.sum(yp)
    x2 = (x-cx)**2
    y2 = (y-cy)**2
    sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
    sy = np.sqrt(np.sum(y2*yp)/np.sum(img))
    x3 = (x-cx)**3
    y3 = (y-cy)**3
    skewx = np.sum(xp*x3)/(np.sum(img) * sx**3)
    skewy = np.sum(yp*y3)/(np.sum(img) * sy**3)
    x4 = (x-cx)**4
    y4 = (y-cy)**4
    kurtx = np.sum(xp*x4)/(np.sum(img) * sx**4) - 3
    kurty = np.sum(yp*y4)/(np.sum(img) * sy**4) - 3

    return (skewx , skewy), (kurtx, kurty)
def Calculating_Centroid(img):
    NumberOfWhites = 0
    a = np.array([0,0])
    for i in range(len(img)):
        for j in range(len(img[0])):
            if img[i][j]==True:
                b = np.array([i,j])
                a = np.add(a,b)
                NumberOfWhites += 1
    rowcols = np.array([img.shape[0], img.shape[1]])
    centroid = a/NumberOfWhites
    centroid = centroid/rowcols
    return centroid[0], centroid[1]
def Feature_Extraction(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    img = PreProccesing(path, display=display)
    ratio = Calculating_Ratio(img)
    centroid = Calculating_Centroid(img)
    eccentricity, solidity = Calculating_EccentricityAndSolidity(img)
    skewness, kurtosis = Calculating_SkewKurtosis(img)
    feature_tuple = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)
    features = (feature_tuple[0], feature_tuple[1][0], feature_tuple[1][1], feature_tuple[2], feature_tuple[3], feature_tuple[4][0], feature_tuple[4][1], feature_tuple[5][0], feature_tuple[5][1])
    return features

# Features Manipulation :

In [6]:
def makeCSV():
    if not(os.path.exists('F:\\Aac dataset\\Dataset\\Features')):
        os.mkdir('F:\\Aac dataset\\Dataset\\Features')
        print('New folder "Features" created')
    if not(os.path.exists('F:\\Aac dataset\\Dataset\\Features/Training')):
        os.mkdir('F:\\Aac dataset\\Dataset\\Features/Training')
        print('New folder "Features/Training" created')
    if not(os.path.exists('F:\\Aac dataset\\Dataset\\Features/Testing')):
        os.mkdir('F:\\Aac dataset\\Dataset\\Features/Testing')
        print('New folder "Features/Testing" created')
    for person in range(1,13):
        per = ('00'+str(person))[-3:]
        print('Saving features for person id-',per)
        
        with open('F:\\Aac dataset\\Dataset\\Features\\Training/training_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            for i in range(0,3):
                source = os.path.join(Orginal_Signatures, per+per+'_00'+str(i)+'.png')
                features = Feature_Extraction(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(0,3):
                source = os.path.join(Fake_Signatures, '021'+per+'_00'+str(i)+'.png')
                features = Feature_Extraction(path=source)
                handle.write(','.join(map(str, features))+',0\n')
        
        with open('F:\\Aac dataset\\Dataset\\Features\\Testing/testing_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            for i in range(3, 5):
                source = os.path.join(Orginal_Signatures, per+per+'_00'+str(i)+'.png')
                features = Feature_Extraction(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(3,5):
                source = os.path.join(Fake_Signatures, '021'+per+'_00'+str(i)+'.png')
                features = Feature_Extraction(path=source)
                handle.write(','.join(map(str, features))+',0\n')

In [7]:
makeCSV()

Saving features for person id- 001


D:\ANACONDA PYTHON\lib\site-packages\skimage\measure\_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
D:\ANACONDA PYTHON\lib\site-packages\skimage\measure\_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


Saving features for person id- 002
Saving features for person id- 003
Saving features for person id- 004
Saving features for person id- 005
Saving features for person id- 006
Saving features for person id- 007
Saving features for person id- 008
Saving features for person id- 009
Saving features for person id- 010
Saving features for person id- 011
Saving features for person id- 012


# Tensor Flow Model :

In [10]:

def WritingFeatures(path):
    ExtractedFeatures = Feature_Extraction(path)
    if not(os.path.exists('F:\\Aac dataset\\Dataset\\TestFeatures')):
        os.mkdir('F:\\Aac dataset\\Dataset\\TestFeatures')
    with open('F:\\Aac dataset\\Dataset\\TestFeatures/testcsv.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
        handle.write(','.join(map(str, ExtractedFeatures))+'\n')
        
n_input = 9
train_person_id = input("Enter person's id : ")
test_image_path = input("Enter path of signature image : ")
train_path = 'F:\\Aac dataset\\Dataset\\Features\\Training/training_'+train_person_id+'.csv'
WritingFeatures(test_image_path)
test_path = 'F:\\Aac dataset\\Dataset\\TestFeatures/testcsv.csv'

def readCSV(train_path, test_path, type2=False):
    df = pd.read_csv(train_path, usecols=range(n_input))
    train_input = np.array(df.values)
    train_input = train_input.astype(np.float32, copy=False)
    df = pd.read_csv(train_path, usecols=(n_input,))
    temp = [elem[0] for elem in df.values]
    correct = np.array(temp)
    corr_train = keras.utils.np_utils.to_categorical(correct,2) 
    df = pd.read_csv(test_path, usecols=range(n_input))
    test_input = np.array(df.values)
    test_input = test_input.astype(np.float32, copy=False)
    if not(type2):
        df = pd.read_csv(test_path, usecols=(n_input,))
        temp = [elem[0] for elem in df.values]
        correct = np.array(temp)
        corr_test = keras.utils.np_utils.to_categorical(correct,2) 
    if not(type2):
        return train_input, corr_train, test_input, corr_test
    else:
        return train_input, corr_train, test_input

ops.reset_default_graph()

learning_rate = 0.001
training_epochs = 1000
display_step = 1

n_hidden_1 = 7 
n_hidden_2 = 10
n_hidden_3 = 30 
n_classes = 2 


X = tf.compat.v1.placeholder(dtype = tf.float32 ,shape= [None, n_input])
Y = tf.compat.v1.placeholder(dtype = tf.float32 ,shape= [None, n_classes])

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], seed=1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes], seed=2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], seed=3)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes], seed=4))
}


def multilayer_perceptron(x):
    layer_1 = tf.tanh((tf.matmul(x, weights['h1']) + biases['b1']))
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    out_layer = tf.tanh(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

logits = multilayer_perceptron(X)


loss_op = tf.reduce_mean(tf.squared_difference(logits, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

pred = tf.nn.softmax(logits) 
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

def evaluate(train_path, test_path, type2=False):   
    if not(type2):
        train_input, corr_train, test_input, corr_test = readCSV(train_path, test_path)
    else:
        train_input, corr_train, test_input = readCSV(train_path, test_path, type2)
    ans = 'Random'
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            _, cost = sess.run([train_op, loss_op], feed_dict={X: train_input, Y: corr_train})
            if cost<0.0001:
                break
        accuracy1 =  accuracy.eval({X: train_input, Y: corr_train})
        if type2 is False:
            accuracy2 =  accuracy.eval({X: test_input, Y: corr_test})
            return accuracy1, accuracy2
        else:
            prediction = pred.eval({X: test_input})
            if prediction[0][1]>prediction[0][0]:
                print('Genuine Image')
                return True
            else:
                print('Forged Image')
                return False



evaluate(train_path, test_path, type2=True) 

Enter person's id : 012
Enter path of signature image : F:\\Aac dataset\\Dataset\\forged\\021012_000.png
Forged Image


False